Import libs

In [1]:
from neo4j import GraphDatabase
import pathlib

Create connection

In [2]:
class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
                
        return response
conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="vovse")
relpath = pathlib.Path().absolute().as_uri()

Import data

In [14]:
# Delete everything in graph
query = '''MATCH (n) DETACH DELETE n'''
out = conn.query(query)

# User data
query = '''
CALL apoc.load.json('%s/../jsondata/0001-users.json') YIELD value
UNWIND value.users AS u
MERGE (domainUser:DomainUser {sid:u.SID})
SET domainUser += u.Properties
''' % relpath
out = conn.query(query)

# Group data - DC local groups
query = '''
CALL apoc.load.json('%s/../jsondata/0001-groups.json') YIELD value
UNWIND value.groups AS g
WITH g WHERE g.type = 'LocalGroup'
MERGE (localGroup:LocalGroup {sid:g.SID, computerName:g.Properties.domain})
SET localGroup += g.Properties
WITH g, localGroup
UNWIND g.members AS mb
FOREACH (_ IN case when mb.type = 'LocalGroup' then [1] else [] end|
	MERGE (localSubGroup:LocalGroup {sid:mb.SID, computerName:g.Properties.domain})
	MERGE (localGroup)-[:hasMember]->(localSubGroup)
)
FOREACH (_ IN case when mb.type = 'DomainUser' then [1] else [] end|
	MERGE (domainUser:DomainUser {sid:mb.SID})
	MERGE (localGroup)-[:hasMember]->(domainUser)
)
FOREACH (_ IN case when mb.type = 'DomainGroup' then [1] else [] end|
	MERGE (domainGroup:DomainGroup {sid:mb.SID})
	MERGE (localGroup)-[:hasMember]->(domainGroup)
)
''' % relpath
out = conn.query(query)

# Group data - domain groups
query = '''
CALL apoc.load.json('%s/../jsondata/0001-groups.json') YIELD value
UNWIND value.groups AS g
WITH g WHERE g.type = 'DomainGroup'
MERGE (domainGroup:DomainGroup {sid:g.SID})
SET domainGroup += g.Properties
WITH g, domainGroup
UNWIND g.members AS mb
FOREACH (_ IN case when mb.type = 'LocalGroup' then [1] else [] end|
    MERGE (localGroup:LocalGroup {sid:mb.SID, computerName:g.Properties.domain})
    MERGE (domainGroup)-[:hasMember]->(localGroup)
)
FOREACH (_ IN case when mb.type = 'DomainUser' then [1] else [] end|
    MERGE (domainUser:DomainUser {sid:mb.SID})
    MERGE (domainGroup)-[:hasMember]->(domainUser)
)
FOREACH (_ IN case when mb.type = 'DomainGroup' then [1] else [] end|
    MERGE (domainSubGroup:DomainGroup {sid:mb.SID})
    MERGE (domainGroup)-[:hasMember]->(domainSubGroup)
)
''' % relpath 
out = conn.query(query)

# Computer data
query = '''
CALL apoc.load.json('%s/../jsondata/0001-computers.json') YIELD value
UNWIND value.computers AS c
MERGE (computer:Computer {sid:c.SID})
SET computer += c.Properties

// Local users
WITH c, computer
UNWIND c.LocalUsers AS lu
MERGE (localUser:LocalUser {name:lu.name, sid:lu.SID, computerName:computer.name})
MERGE (computer)-[:hasUser]->(localUser)

// Local groups
WITH c, computer
UNWIND c.LocalGroups AS lg
MERGE (localGroup:LocalGroup {name:lg.name, sid:lg.SID, computerName:computer.name})
MERGE (computer)-[:hasGroup]->(localGroup)

// Local group members
WITH lg, localGroup, c, computer
UNWIND lg.members AS mb
FOREACH (_ IN case when mb.type = 'LocalGroup' then [1] else [] end|
	MERGE (localSubGroup:LocalGroup {name:mb.name, sid:mb.SID, computerName:computer.name})
	MERGE (computer)-[:hasGroup]->(localSubGroup)
	MERGE (localGroup)-[:hasMember]->(localSubGroup)
)
FOREACH (_ IN case when mb.type = 'LocalUser' then [1] else [] end|
	MERGE (localUser:LocalUser {name:mb.name, sid:mb.SID, computerName:computer.name})
	MERGE (computer)-[:hasUser]->(localUser)
	MERGE (localGroup)-[:hasMember]->(localUser)
)
FOREACH (_ IN case when mb.type = 'DomainUser' then [1] else [] end|
	MERGE (domainUser:DomainUser {sid:mb.SID})
	MERGE (localGroup)-[:hasMember]->(domainUser)
)
FOREACH (_ IN case when mb.type = 'DomainGroup' then [1] else [] end|
	MERGE (domainGroup:DomainGroup {sid:mb.SID})
	MERGE (localGroup)-[:hasMember]->(domainGroup)
)

// URA
WITH c, computer
UNWIND c.URA AS ura
FOREACH (principal IN ura.principals |
    FOREACH (_ IN case when principal.type = 'LocalGroup' then [1] else [] end|
        MERGE (localGroup:LocalGroup {name:principal.name, sid:principal.SID, computerName:computer.name})
        MERGE (localGroup)-[:hasRight {name:ura.right}]->(computer)
    )
    FOREACH (_ IN case when principal.type = 'LocalUser' then [1] else [] end|
        MERGE (localUser:LocalUser {name:principal.name, sid:principal.SID, computerName:computer.name})
        MERGE (localUser)-[:hasRight {name:ura.right}]->(computer)
    )
    FOREACH (_ IN case when principal.type = 'DomainUser' then [1] else [] end|
        MERGE (domainUser:DomainUser {sid:principal.SID})
        MERGE (domainUser)-[:hasRight {name:ura.right}]->(computer)
    )
    FOREACH (_ IN case when principal.type = 'DomainGroup' then [1] else [] end|
        MERGE (domainGroup:DomainGroup {sid:principal.SID})
        MERGE (domainGroup)-[:hasRight {name:ura.right}]->(computer)
    )
)
''' % relpath 
out = conn.query(query)

# Container data
query = '''
CALL apoc.load.json('%s/../jsondata/0001-containers.json') YIELD value
UNWIND value.containers AS c
CREATE (container:Container {GUID:c.ObjectGUID})
SET container += c.Properties
WITH c, container
UNWIND c.Principals AS principal
FOREACH (_ IN case when principal.type = 'LocalUser' then [1] else [] end|
	MERGE (localUser:LocalUser {sid:principal.SID, computerName:c.Properties.domain})
	MERGE (container)-[:contains]->(localUser)
)
FOREACH (_ IN case when principal.type = 'LocalGroup' then [1] else [] end|
	MERGE (localGroup:LocalGroup {sid:principal.SID, computerName:c.Properties.domain})
	MERGE (container)-[:contains]->(localGroup)
)
FOREACH (_ IN case when principal.type = 'DomainUser' then [1] else [] end|
	MERGE (domainUser:DomainUser {sid:principal.SID})
	MERGE (container)-[:contains]->(domainUser)
)
FOREACH (_ IN case when principal.type = 'DomainGroup' then [1] else [] end|
	MERGE (domainGroup:DomainGroup {sid:principal.SID})
	MERGE (container)-[:contains]->(domainGroup)
)
FOREACH (_ IN case when principal.type = 'Computer' then [1] else [] end|
	MERGE (computer:Computer {sid:principal.SID})
	MERGE (container)-[:contains]->(computer)
)
''' % relpath
out = conn.query(query)

# Connect DC groups to DCs
query = '''
MATCH (dc:Computer) WHERE dc.PrimaryGroupSid ENDS WITH '-516'
WITH dc
MATCH (dcGroup:LocalGroup) WHERE dcGroup.domain = dc.domain
MERGE (dc)-[:hasGroup]->(dcGroup)
'''
out = conn.query(query)

Get all data

In [15]:
query = '''MATCH (n) RETURN n'''
out = conn.query(query)

Get domain users and groups which are admin on a host

In [32]:
query = '''
MATCH (adminGrp:LocalGroup)
WHERE adminGrp.sid = 'S-1-5-32-544'
WITH adminGrp
MATCH (adminGrp)-[:hasMember*1..]->(x)
WHERE x:DomainUser OR x:DomainGroup
RETURN DISTINCT x.name AS SamAccountName, adminGrp.computerName AS computer
'''
out = conn.query(query)
out

[<Record SamAccountName='Domain Users' computer='HOT-JUMPHOST.HOT.LOCAL'>]